In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [4]:
def Act(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Act', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def Env(states, actions, state_size, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Env', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        states_logits = tf.layers.dense(inputs=nl2, units=state_size, trainable=False)
        Qlogits = tf.layers.dense(inputs=nl2, units=1, trainable=False)
        return states_logits, Qlogits

In [6]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    ################################################ a = act(s)
    actions_logits = Act(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ################################################ s', r = env(s, a)
    e_next_states_logits, eQs = Env(actions=actions_labels, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size)
    a_next_states_logits, aQs = Env(actions=actions_logits, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size, reuse=True)
    next_states_labels = tf.nn.sigmoid(next_states)
    eloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=e_next_states_logits, # GQN
                                                                   labels=next_states_labels))
    eloss += -tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                     labels=next_states_labels)) # maximize loss
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                    labels=next_states_labels)) # minimize loss
    eQs_logits = tf.reshape(eQs, shape=[-1])
    aQs_logits = tf.reshape(aQs, shape=[-1])
    #################################################### s'', Q' = ~env(s', ~a')
    next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    next_states_logits, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                                   action_size=action_size, state_size=state_size, reuse=True)
    aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=eQs_logits, # GAN
                                                                    labels=rates)) # 0-1 real
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
                                                                     labels=tf.ones_like(rates))) # max
    ###################################################### Q(s,a)= r + Q'(s',a')
    targetQs = rewards + (gamma * aQs2_logits) # DQN/Qlearning
    eloss += tf.reduce_mean(tf.square(eQs_logits - targetQs)) # real
    eloss += tf.reduce_mean(tf.square(aQs_logits - rewards)) # min
    aloss2 += tf.reduce_mean(tf.square(aQs_logits - targetQs)) # max
    eloss += tf.reduce_mean((aQs_logits+aQs2_logits)/2) # min
    aloss2 += -tf.reduce_mean((aQs_logits+aQs2_logits)/2) # max
#     #################################################### s'', Q' = ~env(s', ~a'): Repeatable 1
#     next_actions_logits = Act(states=next_states_logits, hidden_size=hidden_size, action_size=action_size, 
#                               reuse=True)
#     next_states_logits, aQs2 = Env(actions=next_actions_logits, states=next_states_logits, hidden_size=hidden_size, 
#                                    action_size=action_size, state_size=state_size, reuse=True)
#     aQs_logits = aQs2_logits # pervious one
#     aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
# #     eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
# #                                                                     labels=tf.zeros_like(rates))) # min
# #     aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
# #                                                                      labels=tf.ones_like(rates))) # max
#     ###################################################### Q(s,a)= r + Q'(s',a')
#     targetQs = rewards + (gamma * aQs2_logits) # DQN/Qlearning
#     eloss += tf.reduce_mean(tf.square(aQs_logits - rewards)) # min
#     aloss2 += tf.reduce_mean(tf.square(aQs_logits - targetQs)) # max
#     #################################################### s'', Q' = ~env(s', ~a'): Repeatable 2
#     next_actions_logits = Act(states=next_states_logits, hidden_size=hidden_size, action_size=action_size, 
#                               reuse=True)
#     next_states_logits, aQs2 = Env(actions=next_actions_logits, states=next_states_logits, hidden_size=hidden_size, 
#                                    action_size=action_size, state_size=state_size, reuse=True)
#     aQs_logits = aQs2_logits # pervious one
#     aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
# #     eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
# #                                                                     labels=tf.zeros_like(rates))) # min
# #     aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
# #                                                                      labels=tf.ones_like(rates))) # max
#     ###################################################### Q(s,a)= r + Q'(s',a')
#     targetQs = rewards + (gamma * aQs2_logits) # DQN/Qlearning
#     eloss += tf.reduce_mean(tf.square(aQs_logits - rewards)) # min
#     aloss2 += tf.reduce_mean(tf.square(aQs_logits - targetQs)) # max
    return actions_logits, aloss, eloss, aloss2

In [7]:
def model_opt(a_loss, e_loss, a_loss2, a_learning_rate, e_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('Act')]
    e_vars = [var for var in t_vars if var.name.startswith('Env')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        e_opt = tf.train.AdamOptimizer(e_learning_rate).minimize(e_loss, var_list=e_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, e_opt, a_opt2

In [8]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, e_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.e_loss, self.a_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.e_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        e_loss=self.e_loss,
                                                        a_loss2=self.a_loss2, 
                                                        a_learning_rate=a_learning_rate,
                                                        e_learning_rate=e_learning_rate)

In [9]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch
#     def sample(self, batch_size):
#         idx = np.random.choice(np.arange(len(self.buffer)), size=batch_size, replace=False)
#         return [self.buffer[ii] for ii in idx]

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [10]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [11]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
e_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size
gamma=0.99

In [12]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size, gamma=gamma,
              a_learning_rate=a_learning_rate, e_learning_rate=e_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [13]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        state = env.reset()
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
aloss_list, eloss_list, aloss2_list = [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes for running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        aloss_batch, eloss_batch, aloss2_batch = [], [], []
        total_reward = 0
        state = env.reset()
        num_step = 0
        rate = -1

        # Training steps/batches
        while True:
            # Explore (env) or Exploit (model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            while True:
                idx = np.random.choice(np.arange(memory_size// batch_size))
                states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                states = states[rates >= np.max(rates)]
                actions = actions[rates >= np.max(rates)]
                next_states = next_states[rates >= np.max(rates)]
                rewards = rewards[rates >= np.max(rates)]
                dones = dones[rates >= np.max(rates)]
                rates = rates[rates >= np.max(rates)]
                if np.count_nonzero(dones) > 0 and len(dones) > 1 and np.max(rates) > 0:
                    break
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            eloss, _ = sess.run([model.e_loss, model.e_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss2, _= sess.run([model.a_loss2, model.a_opt2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            # print(len(dones), np.count_nonzero(dones), np.max(rates))
            aloss_batch.append(aloss)
            eloss_batch.append(eloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        # Rating the latest played episode
        rate = total_reward/500 # update rate at the end/ when episode is done
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                memory.buffer[-1-idx][-1] = rate # rate the trajectory/data

        # Print out
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'eloss:{:.4f}'.format(np.mean(eloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        aloss_list.append([ep, np.mean(aloss_batch)])
        eloss_list.append([ep, np.mean(eloss_batch)])
        aloss2_list.append([ep, np.mean(aloss2_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:18.0000 R:18.0000 rate:0.0360 aloss:0.7064 eloss:4.1539 aloss2:2.5974 exploreP:0.9982
Episode:1 meanR:22.5000 R:27.0000 rate:0.0540 aloss:0.7031 eloss:4.2166 aloss2:2.7174 exploreP:0.9956
Episode:2 meanR:23.3333 R:25.0000 rate:0.0500 aloss:0.6988 eloss:4.1235 aloss2:2.5819 exploreP:0.9931
Episode:3 meanR:20.7500 R:13.0000 rate:0.0260 aloss:0.7109 eloss:4.1038 aloss2:2.5301 exploreP:0.9918
Episode:4 meanR:20.4000 R:19.0000 rate:0.0380 aloss:0.6985 eloss:4.0859 aloss2:2.6038 exploreP:0.9900
Episode:5 meanR:23.0000 R:36.0000 rate:0.0720 aloss:0.7015 eloss:4.0721 aloss2:2.6061 exploreP:0.9864
Episode:6 meanR:21.4286 R:12.0000 rate:0.0240 aloss:0.6944 eloss:4.0545 aloss2:2.6195 exploreP:0.9853
Episode:7 meanR:21.2500 R:20.0000 rate:0.0400 aloss:0.6967 eloss:4.0358 aloss2:2.5922 exploreP:0.9833
Episode:8 meanR:20.3333 R:13.0000 rate:0.0260 aloss:0.6924 eloss:4.0289 aloss2:2.5882 exploreP:0.9820
Episode:9 meanR:21.0000 R:27.0000 rate:0.0540 aloss:0.6969 eloss:4.0078 aloss2:2.6

Episode:80 meanR:22.4691 R:20.0000 rate:0.0400 aloss:0.6872 eloss:3.2642 aloss2:2.2792 exploreP:0.8353
Episode:81 meanR:22.3902 R:16.0000 rate:0.0320 aloss:0.6838 eloss:3.2574 aloss2:2.2565 exploreP:0.8339
Episode:82 meanR:22.5663 R:37.0000 rate:0.0740 aloss:0.6879 eloss:3.2619 aloss2:2.2643 exploreP:0.8309
Episode:83 meanR:22.4881 R:16.0000 rate:0.0320 aloss:0.6878 eloss:3.2539 aloss2:2.2482 exploreP:0.8296
Episode:84 meanR:22.7176 R:42.0000 rate:0.0840 aloss:0.6842 eloss:3.2584 aloss2:2.2643 exploreP:0.8262
Episode:85 meanR:22.6512 R:17.0000 rate:0.0340 aloss:0.6860 eloss:3.2585 aloss2:2.2645 exploreP:0.8248
Episode:86 meanR:22.7241 R:29.0000 rate:0.0580 aloss:0.6853 eloss:3.2530 aloss2:2.2600 exploreP:0.8224
Episode:87 meanR:22.7955 R:29.0000 rate:0.0580 aloss:0.6844 eloss:3.2459 aloss2:2.2526 exploreP:0.8201
Episode:88 meanR:22.8989 R:32.0000 rate:0.0640 aloss:0.6870 eloss:3.2499 aloss2:2.2454 exploreP:0.8175
Episode:89 meanR:22.9000 R:23.0000 rate:0.0460 aloss:0.6814 eloss:3.2498 

Episode:159 meanR:35.7800 R:39.0000 rate:0.0780 aloss:0.6869 eloss:3.2644 aloss2:2.2629 exploreP:0.6167
Episode:160 meanR:36.1700 R:65.0000 rate:0.1300 aloss:0.7038 eloss:3.2637 aloss2:2.2683 exploreP:0.6128
Episode:161 meanR:37.2200 R:129.0000 rate:0.2580 aloss:0.6904 eloss:3.2701 aloss2:2.2617 exploreP:0.6050
Episode:162 meanR:38.5100 R:145.0000 rate:0.2900 aloss:0.6993 eloss:3.2644 aloss2:2.2555 exploreP:0.5965
Episode:163 meanR:38.9000 R:78.0000 rate:0.1560 aloss:0.7551 eloss:3.2658 aloss2:2.2441 exploreP:0.5919
Episode:164 meanR:39.0500 R:31.0000 rate:0.0620 aloss:0.6994 eloss:3.2590 aloss2:2.2594 exploreP:0.5901
Episode:165 meanR:39.0700 R:56.0000 rate:0.1120 aloss:0.7180 eloss:3.2692 aloss2:2.2447 exploreP:0.5869
Episode:166 meanR:40.1300 R:123.0000 rate:0.2460 aloss:0.6973 eloss:3.2570 aloss2:2.2517 exploreP:0.5798
Episode:167 meanR:41.2000 R:121.0000 rate:0.2420 aloss:0.7072 eloss:3.2658 aloss2:2.2319 exploreP:0.5730
Episode:168 meanR:42.0200 R:104.0000 rate:0.2080 aloss:0.696

Episode:238 meanR:134.5200 R:261.0000 rate:0.5220 aloss:0.6869 eloss:3.1036 aloss2:2.3332 exploreP:0.1891
Episode:239 meanR:138.3900 R:483.0000 rate:0.9660 aloss:0.6980 eloss:3.0980 aloss2:2.3360 exploreP:0.1807
Episode:240 meanR:139.9700 R:201.0000 rate:0.4020 aloss:0.6906 eloss:3.0994 aloss2:2.3369 exploreP:0.1773
Episode:241 meanR:142.6000 R:302.0000 rate:0.6040 aloss:0.6876 eloss:3.1031 aloss2:2.3396 exploreP:0.1723
Episode:242 meanR:144.2700 R:237.0000 rate:0.4740 aloss:0.6876 eloss:3.0950 aloss2:2.3555 exploreP:0.1685
Episode:243 meanR:146.4800 R:251.0000 rate:0.5020 aloss:0.6939 eloss:3.1035 aloss2:2.3441 exploreP:0.1646
Episode:244 meanR:147.9500 R:208.0000 rate:0.4160 aloss:0.6914 eloss:3.0912 aloss2:2.3563 exploreP:0.1614
Episode:245 meanR:149.4100 R:201.0000 rate:0.4020 aloss:0.6828 eloss:3.1059 aloss2:2.3491 exploreP:0.1584
Episode:246 meanR:151.7200 R:262.0000 rate:0.5240 aloss:0.6880 eloss:3.0935 aloss2:2.3577 exploreP:0.1546
Episode:247 meanR:153.9000 R:232.0000 rate:0.4

Episode:316 meanR:214.2600 R:234.0000 rate:0.4680 aloss:0.6632 eloss:3.0830 aloss2:2.5612 exploreP:0.0465
Episode:317 meanR:214.8000 R:156.0000 rate:0.3120 aloss:0.6676 eloss:3.0785 aloss2:2.5539 exploreP:0.0459
Episode:318 meanR:214.4300 R:160.0000 rate:0.3200 aloss:0.6647 eloss:3.0864 aloss2:2.5636 exploreP:0.0453
Episode:319 meanR:215.5500 R:173.0000 rate:0.3460 aloss:0.6589 eloss:3.0924 aloss2:2.5695 exploreP:0.0447
Episode:320 meanR:215.5800 R:246.0000 rate:0.4920 aloss:0.6459 eloss:3.0881 aloss2:2.5652 exploreP:0.0439
Episode:321 meanR:214.9200 R:183.0000 rate:0.3660 aloss:0.6433 eloss:3.0878 aloss2:2.5578 exploreP:0.0433
Episode:322 meanR:211.5300 R:161.0000 rate:0.3220 aloss:0.6636 eloss:3.0835 aloss2:2.5644 exploreP:0.0427
Episode:323 meanR:213.4600 R:216.0000 rate:0.4320 aloss:0.6494 eloss:3.0923 aloss2:2.5740 exploreP:0.0420
Episode:324 meanR:214.7400 R:180.0000 rate:0.3600 aloss:0.6679 eloss:3.0857 aloss2:2.5793 exploreP:0.0415
Episode:325 meanR:213.9900 R:159.0000 rate:0.3

...

Episode:706 meanR:217.0200 R:206.0000 rate:0.4120 aloss:0.3553 eloss:2.9276 aloss2:3.1768 exploreP:0.0100
Episode:707 meanR:216.6400 R:181.0000 rate:0.3620 aloss:0.3441 eloss:2.9464 aloss2:3.1675 exploreP:0.0100
Episode:708 meanR:215.9100 R:294.0000 rate:0.5880 aloss:0.3315 eloss:2.9544 aloss2:3.1665 exploreP:0.0100
Episode:709 meanR:215.8300 R:186.0000 rate:0.3720 aloss:0.3450 eloss:2.9414 aloss2:3.2040 exploreP:0.0100
Episode:710 meanR:216.9200 R:313.0000 rate:0.6260 aloss:0.3358 eloss:2.9473 aloss2:3.1676 exploreP:0.0100
Episode:711 meanR:217.3100 R:208.0000 rate:0.4160 aloss:0.3397 eloss:2.9335 aloss2:3.1817 exploreP:0.0100
Episode:712 meanR:217.1200 R:167.0000 rate:0.3340 aloss:0.3346 eloss:2.9643 aloss2:3.1911 exploreP:0.0100
Episode:713 meanR:217.0900 R:189.0000 rate:0.3780 aloss:0.3367 eloss:2.9221 aloss2:3.1549 exploreP:0.0100
Episode:714 meanR:216.1500 R:227.0000 rate:0.4540 aloss:0.3404 eloss:2.9244 aloss2:3.1799 exploreP:0.0100
Episode:715 meanR:216.1000 R:214.0000 rate:0.4

Episode:784 meanR:208.5400 R:194.0000 rate:0.3880 aloss:0.3086 eloss:2.6501 aloss2:2.9852 exploreP:0.0100
Episode:785 meanR:208.6800 R:207.0000 rate:0.4140 aloss:0.3065 eloss:2.6333 aloss2:2.9833 exploreP:0.0100
Episode:786 meanR:208.7200 R:203.0000 rate:0.4060 aloss:0.3101 eloss:2.6498 aloss2:2.9587 exploreP:0.0100
Episode:787 meanR:209.2200 R:231.0000 rate:0.4620 aloss:0.3103 eloss:2.6411 aloss2:2.9820 exploreP:0.0100
Episode:788 meanR:208.7900 R:202.0000 rate:0.4040 aloss:0.3149 eloss:2.6625 aloss2:2.9540 exploreP:0.0100
Episode:789 meanR:208.1700 R:186.0000 rate:0.3720 aloss:0.3083 eloss:2.6572 aloss2:2.8961 exploreP:0.0100
Episode:790 meanR:208.3100 R:224.0000 rate:0.4480 aloss:0.3035 eloss:2.6083 aloss2:2.9292 exploreP:0.0100
Episode:791 meanR:208.0700 R:193.0000 rate:0.3860 aloss:0.3069 eloss:2.6132 aloss2:2.8963 exploreP:0.0100
Episode:792 meanR:209.2300 R:294.0000 rate:0.5880 aloss:0.3040 eloss:2.6026 aloss2:2.9036 exploreP:0.0100
Episode:793 meanR:209.2400 R:173.0000 rate:0.3

Episode:862 meanR:220.5600 R:452.0000 rate:0.9040 aloss:0.2911 eloss:2.1730 aloss2:3.4261 exploreP:0.0100
Episode:863 meanR:221.5300 R:270.0000 rate:0.5400 aloss:0.2939 eloss:2.1626 aloss2:3.4275 exploreP:0.0100
Episode:864 meanR:221.9100 R:223.0000 rate:0.4460 aloss:0.2906 eloss:2.1450 aloss2:3.5005 exploreP:0.0100
Episode:865 meanR:221.9700 R:198.0000 rate:0.3960 aloss:0.2910 eloss:2.1630 aloss2:3.4746 exploreP:0.0100
Episode:866 meanR:221.8700 R:211.0000 rate:0.4220 aloss:0.2890 eloss:2.1086 aloss2:3.4574 exploreP:0.0100
Episode:867 meanR:221.2300 R:192.0000 rate:0.3840 aloss:0.2893 eloss:2.0864 aloss2:3.4970 exploreP:0.0100
Episode:868 meanR:220.8900 R:179.0000 rate:0.3580 aloss:0.2901 eloss:2.1187 aloss2:3.4973 exploreP:0.0100
Episode:869 meanR:220.8500 R:183.0000 rate:0.3660 aloss:0.2891 eloss:2.1496 aloss2:3.5707 exploreP:0.0100
Episode:870 meanR:220.4400 R:225.0000 rate:0.4500 aloss:0.2885 eloss:2.1026 aloss2:3.5601 exploreP:0.0100
Episode:871 meanR:220.6500 R:205.0000 rate:0.4

Episode:940 meanR:241.9400 R:500.0000 rate:1.0000 aloss:0.2994 eloss:1.6554 aloss2:3.9608 exploreP:0.0100
Episode:941 meanR:242.1500 R:270.0000 rate:0.5400 aloss:0.3022 eloss:1.6828 aloss2:3.9163 exploreP:0.0100
Episode:942 meanR:242.7300 R:282.0000 rate:0.5640 aloss:0.3014 eloss:1.6835 aloss2:3.9093 exploreP:0.0100
Episode:943 meanR:241.7700 R:278.0000 rate:0.5560 aloss:0.3042 eloss:1.6784 aloss2:3.9081 exploreP:0.0100
Episode:944 meanR:240.9500 R:203.0000 rate:0.4060 aloss:0.3030 eloss:1.6513 aloss2:3.9271 exploreP:0.0100
Episode:945 meanR:241.5600 R:302.0000 rate:0.6040 aloss:0.3034 eloss:1.6540 aloss2:3.9185 exploreP:0.0100
Episode:946 meanR:240.7900 R:193.0000 rate:0.3860 aloss:0.3055 eloss:1.6542 aloss2:3.9073 exploreP:0.0100
Episode:947 meanR:241.7400 R:305.0000 rate:0.6100 aloss:0.3059 eloss:1.6572 aloss2:3.9229 exploreP:0.0100
Episode:948 meanR:243.2200 R:354.0000 rate:0.7080 aloss:0.3035 eloss:1.6267 aloss2:3.9256 exploreP:0.0100
Episode:949 meanR:243.4400 R:201.0000 rate:0.4

Episode:1018 meanR:261.4200 R:202.0000 rate:0.4040 aloss:0.3225 eloss:1.0101 aloss2:4.4913 exploreP:0.0100
Episode:1019 meanR:262.5200 R:342.0000 rate:0.6840 aloss:0.3216 eloss:0.9677 aloss2:4.5814 exploreP:0.0100
Episode:1020 meanR:262.7400 R:256.0000 rate:0.5120 aloss:0.3206 eloss:0.9802 aloss2:4.5266 exploreP:0.0100
Episode:1021 meanR:263.2100 R:226.0000 rate:0.4520 aloss:0.3156 eloss:0.9478 aloss2:4.7306 exploreP:0.0100
Episode:1022 meanR:264.4400 R:310.0000 rate:0.6200 aloss:0.3216 eloss:0.9172 aloss2:4.6050 exploreP:0.0100
Episode:1023 meanR:264.4900 R:278.0000 rate:0.5560 aloss:0.3136 eloss:0.8073 aloss2:4.7381 exploreP:0.0100
Episode:1024 meanR:264.0400 R:221.0000 rate:0.4420 aloss:0.3142 eloss:0.8242 aloss2:4.7596 exploreP:0.0100
Episode:1025 meanR:264.5000 R:244.0000 rate:0.4880 aloss:0.3142 eloss:0.8296 aloss2:4.8059 exploreP:0.0100
Episode:1026 meanR:265.9700 R:367.0000 rate:0.7340 aloss:0.3172 eloss:0.7862 aloss2:4.7477 exploreP:0.0100
Episode:1027 meanR:266.2700 R:282.000

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(aloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses')

In [ ]:
eps, arr = np.array(eloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Env losses')

In [ ]:
eps, arr = np.array(aloss2_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses 2')

## Testing

Let's checkout how our trained agent plays the game.

In [21]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
